In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

df= pd.DataFrame(columns=['Türkçe', 'İngilizce','Konu','Ses','okumuş','ornekler'])

response = requests.get('https://evdeingilizcem.com/ingilizce-kelimeler/').content.decode('utf-8')

soup = BeautifulSoup(response, 'html.parser')

clas1= soup.find('div', class_='entry-content entry clearfix')

tbody = clas1.find('ul')

a_tags = tbody.find_all('li')

for i in a_tags[1:-5]:
    a = i.find('a')
    konu = a.text.replace('İngilizce', '').split('seviye')[0][2:-1]
    
    
    if konu == 'ülkeler, milletler ve diller' or  konu == 'günler, aylar, mevsimler':
        continue
    print(f'Konu:_{konu}_')
    
    herf = a.get('href')
    response2 = requests.get(herf).content.decode('utf-8')
    soup2 = BeautifulSoup(response2, 'html.parser')
    table = soup2.find('tbody')

    for ii in table.find_all('tr'):
        tds = ii.find_all('td')
        new_row = {'Türkçe': tds[1].text, 'İngilizce': tds[0].text, 'Konu': konu}
        df = pd.concat([df, pd.DataFrame(new_row, index=[0])], ignore_index=True)

Konu:_hava durumu kelimeleri_
Konu:_aile üyeleri_
Konu:_meyveler_
Konu:_sebzeler_
Konu:_evcil hayvan isimleri_
Konu:_vahşi hayvan isimleri_
Konu:_evin bölümleri ve ev çeşitleri_
Konu:_meslekler_
Konu:_vücudun bölümleri_
Konu:_hobiler_
Konu:_duygu ve hisler_
Konu:_seyahat kelimeleri_
Konu:_okul araç gereç kelimeleri_


In [3]:
import json
import requests
import pandas as pd

# Assuming df is your DataFrame and it's already defined somewhere above this script
for index in df.index:
    word = df.loc[index, 'İngilizce']
    try:
        response = requests.get(f"https://api.dictionaryapi.dev/api/v2/entries/en/{word}")
        response.raise_for_status()
        json_data = response.json()
    except (requests.RequestException, json.JSONDecodeError):
        df.drop(index, inplace=True)
        continue

    try:
        phonetics = json_data[0]['phonetics']
        if phonetics and phonetics[0]['audio']:
            df.at[index, 'Ses'] = phonetics[0]['audio']
            df.at[index, 'okumuş'] = phonetics[0]['text']
            
            meanings = json_data[0]['meanings']
            df.at[index, 'ornekler'] = json.dumps(meanings)  # Storing meanings as a JSON string
            
            print(index, end=' ')
        else:
            df.drop(index, inplace=True)
    except (IndexError, KeyError, TypeError):
        df.drop(index, inplace=True)


0 1 2 4 5 6 7 8 12 19 23 25 30 32 50 55 56 60 64 65 66 68 70 73 76 78 84 87 88 91 96 103 110 111 113 115 116 118 119 120 128 133 134 135 137 138 140 141 142 143 145 150 152 154 156 157 162 163 165 168 174 178 179 180 182 193 197 200 204 205 207 208 210 212 214 217 224 225 229 231 237 240 242 250 251 253 255 256 257 259 261 262 263 265 269 270 272 274 275 276 277 279 282 284 285 286 287 288 289 290 293 301 310 316 317 319 320 321 322 323 324 325 327 329 330 332 334 337 341 344 345 346 347 348 349 350 356 357 361 364 367 368 369 376 378 380 389 391 394 395 396 406 409 410 411 415 417 427 428 431 432 433 435 441 445 448 452 453 454 459 467 468 470 472 477 478 480 481 486 487 488 489 490 493 497 498 502 505 507 508 512 513 515 521 523 524 526 527 530 531 534 535 537 538 540 541 542 545 546 549 552 554 558 560 565 568 572 576 578 582 583 586 591 594 597 598 599 601 604 606 609 610 612 618 619 624 627 632 634 636 640 643 646 649 653 654 655 656 661 662 664 666 667 669 670 672 

In [4]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Muha.kaplan63",
    database="proje"

)
mycursor = mydb.cursor()

In [5]:
mycursor.execute("""CREATE TABLE kelimeler (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Türkçe VARCHAR(255),
    İngilizce VARCHAR(255),
    Konu VARCHAR(255),
    Ses VARCHAR(255),
    okumuş VARCHAR(100),
    ornekler text)""")

In [6]:
import json
for i in df.index:
    if len(df.loc[i]['ornekler'])>5000:
        continue
    sql = "INSERT INTO kelimeler (turkce, ing, konu, ses, okumuş, ornekler) VALUES (%s, %s, %s, %s, %s, %s)"
    val = (df.loc[i]['Türkçe'], df.loc[i]['İngilizce'], df.loc[i]['Konu'], df.loc[i]['Ses'], df.loc[i]['okumuş'], json.dumps(df.loc[i]['ornekler']))
    mycursor.execute(sql, val)
    mydb.commit()